# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [36]:
cities_df = pd.read_csv('../output_data/cities.csv', index_col = False)
cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
gmaps.configure(api_key=g_key)

In [38]:
locations = cities_df[["Lat","Lng"]]
humidity = cities_df["Humidity"]

In [39]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 3

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
weatherconditions_df = cities_df[(cities_df['Max Temp'] >= 75) & (cities_df['Max Temp'] <= 90)]
weatherconditions_df = weatherconditions_df[weatherconditions_df['Wind Speed'] <= 10]
weatherconditions_df = weatherconditions_df[weatherconditions_df['Cloudiness'] <= 3]
weatherconditions_df = weatherconditions_df[weatherconditions_df['Humidity'] <= 70]
hotel_df = weatherconditions_df
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17
316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17
416,416,kirkuk,0,IQ,1558378874,16,35.47,44.40,86.82,7.36
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07
518,518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72
525,525,mecca,1,SA,1558378872,48,21.43,39.83,88.62,1.74


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_df['Hotel Name'] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36,
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17,
316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17,
416,416,kirkuk,0,IQ,1558378874,16,35.47,44.40,86.82,7.36,
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07,
518,518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72,
525,525,mecca,1,SA,1558378872,48,21.43,39.83,88.62,1.74,


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [43]:
# Add marker layer ontop of heat map
# Display figure
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
print(fig)


Figure(layout=FigureLayout(height='420px'))
